# Parallel loops through numpy arrays
We look at speeding up loops through numpy arrays. In this example we have to call a third-party library in each iteration and this third-party library will only accept a subset of our total array. As we are calling a third-party library we can't apply tricks like JIT compilation.

The scenario here is that we have a 3-dimensional array with dimensions (x,y,time). We will imagine that this is a time series of 2-dimensional maps of ocean salinity. Our third-party library is the seawater library. This seawater library only accepts 2-dimensional inputs so we need to loop through the time dimension and call this library on each iteration. 

# Libraries
In this example we will use the built-in [Concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html), [Joblib](https://joblib.readthedocs.io/en/latest/) and [Dask](https://docs.dask.org/en/stable/) libraries.  In the case of Dask we are using the dask delayed API for parallelising the loop.

# Tl;dr
I set out recipes for running in parallel with three libraries. Comparing performance I show that for this task threading performs faster than multiprocessing. I finish up by looking at how the libraries differ in terms of how they handle exceptions.

In [36]:
import numpy as np
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor,ThreadPoolExecutor

from joblib import Parallel,delayed
import dask
import dask.array as da

In [37]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# Generate data
We generate the numpy array we're going to loop through. 

We use a three-dimensional array where we'll think of the dimensions as being `(x,y,time)`. The function that we're calling, however, can only accept two-dimensional inputs in `(x,y)` so we will loop through the `time` dimension.

In [2]:
def generateData(xyLength:int,timesteps:int):
    arr = np.random.standard_normal(size=(xyLength,xyLength,timesteps))
    return arr

arraySmall = generateData(xyLength=3,timesteps=3)    

We define the function that we are going to call in each iteration `timestepFunc`

In [3]:
def timestepFunc(arrTimestep:np.ndarray,timeIndex:int):
    return np.exp(arrTimestep),timeIndex

#### First we create a baseline non-parallelised function to do sequential processing

In [4]:
def serialProcessing(arr:np.ndarray):
    return np.stack(
        [timestepFunc(arrTimestep=arr[:,:,timestep],timeIndex=timestep)[0] for timestep in range(arr.shape[2])],
        axis=2)

# Call the function
outputSerial = serialProcessing(arr=arraySmall)
# Check the outputs are what we expect
np.testing.assert_array_equal(outputSerial,np.exp(arraySmall))

#### The outputs of parallel functions are not in the same order as the inputs

#### Before we create parallel functions we define a helper function that will sort the list of outputs using the time index variable we added to `timestepFunc`.

In [5]:
def sortResults(resultList:list):
    resultList = sorted(resultList,key=lambda x:x[1])
    resultList = [el[0] for el in resultList]
    return resultList

# Parallel processing

## Concurrent.futures

##### The built-in concurrent.futures module is a great place to start with parallel processing. It comes with both a threading and multiprocessing backend. The APIs for these backends are similar, so it's also easy to swap them out and compare them.

In [98]:
from typing import Callable

In [102]:
def concurrentProcessing(arr:np.ndarray,executor,func:Callable):
    """
    Iterate through the array `arr` in parallel with either the threading executor or the multiprocessing executor
    """
#     Create the list that will hold the results of each iteration
    resultList = []
#     Set up the executor 
#   we use a with statement here to ensure the pool of threads/processes gets closed whether the jobs run successfully or not
    with executor() as pool:
#         Loop through the array and store the `futures` that we get from each iteration
        futuresList = [
             pool.submit(
                func,
              arr[:,:,timestep],              
                 timestep
                ) for timestep in range(arr.shape[2])]
#         Gather up the completed tasks
        done_results = concurrent.futures.as_completed(futuresList)
#     Create the list of results
        for _ in futuresList: 
            resultList.append(next(done_results).result())
#         Sort the results back into their original order
        resultList = sortResults(resultList=resultList)
#     Convert the list of results back into a three-dimensional numpy array
    return np.stack(resultList,axis=2)

# Run the function with the multiprocessing `ProcessPoolExecutor` and check that the outputs are the same as for the serial processing
outputConcurrent = concurrentProcessing(arr=arraySmall,executor=ProcessPoolExecutor,func=timestepFunc)
np.testing.assert_array_equal(outputSerial,outputConcurrent)

## Joblib
The joblib library can be either a reimplementation of the built-in `multiprocessing` and `threading` libraries or a wrapper for them. There are some other differences such as:
- a different way of writing the code that you might find more readable
- you can call ctrl-c (= hitting the stop button in a notebook) to interrupt execution of the parallel jobs
- ability to use shared memory for large numpy arrays

There has also been strong co-development of `joblib` and `scikit-learn` so `joblib` is often a good choice for parallelising machine learning workflows.

In [34]:
def joblibProcessing(arr:np.ndarray,backend = "threading",maxNbytes=1,nJobs:int=-1):
#   Iterate through the third-dimension of the array in parallel
    resultList = Parallel(backend=backend,max_nbytes=maxNbytes,n_jobs=nJobs)(delayed(timestepFunc)(arr[:,:,timestep],timestep) for timestep in range(arr.shape[2]))
#   Sort the results back into their original order
    resultList = sortResults(resultList=resultList)
#     Convert the list of results back into a three-dimensional numpy array
    return np.stack(resultList,axis=2)

# Run the function with threading and check that the outputs are the same as for the serial processing
outputJoblib = joblibProcessing(arr=arraySmall)
np.testing.assert_array_equal(outputSerial,outputJoblib)

# Dask delayed
Dask delayed wraps the `concurrent.futures` threading and multiprocessing pools we say above. As with `joblib` you can use ctrl+c to kill any subprocesses. 

In [35]:
def daskDelayedProcessing(arr:np.ndarray):
#   Iterate through the third-dimension of the array in parallel
    resultList = [dask.delayed(timestepFunc,pure=False)(arr[:,:,timestep],timestep) for timestep in range(arr.shape[2])]
#   Trigger computation of the outputs
    resultList = dask.compute(*resultList)
#   Sort the results back into their original order
    resultList = sortResults(resultList=resultList)
#     Convert the list of results back into a three-dimensional numpy array
    return np.stack(resultList,axis=2)
outputDask = daskDelayedProcessing(arr=arraySmall)
np.testing.assert_array_equal(outputSerial,outputDask)

# Timings

##### We generate a larger array here to compare performance. The numbers here should be taken with a pinch of salt - the main point is to show you how to implement these approaches so you can test the results on your own problem

In [39]:
xyLength = 200
timesteps = 2000
arrayLarge = generateData(xyLength=xyLength,timesteps=timesteps)    

In [96]:
printmd("**Serial processing**")
%timeit -n 1 -r 3 serialProcessing(arr=arrayLarge)
printmd("**Multiprocessing**")
printmd("**Multiprocessing with Concurrent.futures**")
%timeit -n 1 -r 3 concurrentProcessing(arr=arrayLarge,executor=ProcessPoolExecutor)
printmd("**Loky multiprocessing with Joblib**")
%timeit -n 1 -r 3 joblibProcessing(arr=arrayLarge,backend="loky")
printmd("**Multiprocessing with Joblib**")
%timeit -n 1 -r 3 joblibProcessing(arr=arrayLarge,backend="multiprocessing")
printmd("**Multiprocessing with Dask**")
dask.config.set(scheduler='processes')  # overwrite default with processes scheduler
%timeit -n 1 -r 3 daskDelayedProcessing(arr=arrayLarge)
printmd("**Threading**")
printmd("**Threading with Concurrent.futures**")
%timeit -n 1 -r 3 concurrentProcessing(arr=arrayLarge,executor=ThreadPoolExecutor)
printmd("**Threading with Joblib**")
%timeit -n 1 -r 3 joblibProcessing(arr=arrayLarge,backend="threading")

printmd("**Threading with Dask**")
dask.config.set(scheduler='threads')  # overwrite default with threads scheduler
%timeit -n 1 -r 3 daskDelayedProcessing(arr=arrayLarge)

**Serial processing**

5.55 s ± 974 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


**Multiprocessing**

**Multiprocessing with Concurrent.futures**

6.76 s ± 40.9 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


...

2.51 s ± 217 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


**Threading with Joblib**

2.55 s ± 146 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


**Threading with Dask**

4.25 s ± 1.09 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Standard numpy
Our `timestepFunc` is a stand-in for a third-party library where we can't work on the entire array in one go. But in our dummy example we can also compare the performance of the underlying numpy function.

In [92]:
printmd("**Standard numpy**")
%timeit -n 1 -r 25 np.exp(arrayLarge)

**Standard numpy**

1.05 s ± 21.1 ms per loop (mean ± std. dev. of 25 runs, 1 loop each)


The time taken on my computer is just half the time taken by the fastest parallel loop. This shows that you need to set standard numpy as your baseline for comparison in cases where it is possible to process the array directly with numpy. 

If we wanted an easier way to parallelise this operation than writing explicit loops, we could use a library with built-in parallelisation like numExpr or Dask. I will explore these approaches in more detail in other notebooks.

In [94]:
printmd("**NumExpr**")
import numexpr as ne
%timeit -n 1 -r 5 ne.evaluate("exp(arrayLarge)")

**NumExpr**

305 ms ± 30.9 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [93]:
printmd("**Dask Array**")
daskArray = da.from_array(arrayLarge)
%timeit -n 1 -r 5 da.exp(daskArray).compute()

**Dask Array**

553 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Differences between the parallel libraries: exception handling

If each of the parallel libraries wraps the built-in `multiprocessing` and `threading` modules you might ask what the difference between them is. We saw above that one difference is that `joblib` and `dask` are more responsive to stop signals from ctrl+c (the stop button in a notebook).

Another important difference is how they **respond to an exception being raised** in a child process. When building automated applications to run for my clients I generally want everything to stop if an error occurs rather than some processes failing and some continuing. We can test how these libraries behave by defining an alternative function that will raise a `ValueError` (giving a big pink box in jupyter) instead of running to completion. In the following cell we'll define that function and call it to make sure it raise the exception:

In [58]:
def timestepRaiseExceptionFunc(arrTimestep:np.ndarray,timeIndex:int):
    raise ValueError("Throw a ValueError to see how the libraries respond") 

timestepRaiseExceptionFunc(arrTimestep=arraySmall[:,:,0],timeIndex=0)

ValueError: Throw a ValueError to see how the libraries respond

### Concurrent.futures

In [95]:
def concurrentProcessingRaiseException(arr:np.ndarray,executor):
    """
    Iterate through the array `arr` in parallel with either the threading executor or the multiprocessing executor
     """
#   Create the list that will hold the results of each iteration
    executor = ThreadPoolExecutor
    resultList = []
    arr = arraySmall
    #     Set up the executor 
    #   we use a with statement here to ensure the pool of threads/processes gets closed whether the jobs run successfully or not
    with executor() as pool:
    #         Loop through the array and store the `futures` that we get from each iteration
        futuresList = [
             pool.submit(
                timestepRaiseExceptionFunc,
              arr[:,:,timestep],              
                 timestep
                ) for timestep in range(arr.shape[2])]
    #         Gather up the completed tasks
        done_results = concurrent.futures.as_completed(futuresList)

concurrentProcessingRaiseException(arr=arraySmall,executor=ThreadPoolExecutor)
concurrentProcessingRaiseException(arr=arraySmall,executor=ProcessPoolExecutor)

Uh oh: we didn't get an exception here. Let's see what happens with joblib and dask

### Joblib

In [57]:
def joblibProcessingRaiseException(arr:np.ndarray,backend = "threading",maxNbytes=1,nJobs:int=-1):
#   Iterate through the third-dimension of the array in parallel
    resultList = Parallel(backend=backend,max_nbytes=maxNbytes,n_jobs=nJobs)(delayed(timestepRaiseExceptionFunc)(arr[:,:,timestep],timestep) for timestep in range(arr.shape[2]))
#   Sort the results back into their original order
    resultList = sortResults(resultList=resultList)
#     Convert the list of results back into a three-dimensional numpy array
    return np.stack(resultList,axis=2)


joblibProcessingRaiseException(arr=arraySmall)

ValueError: Throw a ValueError to see how the libraries respond

### Dask

In [52]:
def daskDelayedProcessingRaiseException(arr:np.ndarray):
#   Iterate through the third-dimension of the array in parallel
    resultList = [dask.delayed(timestepRaiseExceptionFunc,pure=False)(arr[:,:,timestep],timestep) for timestep in range(arr.shape[2])]
#   Trigger computation of the outputs
    resultList = dask.compute(*resultList)
#   Sort the results back into their original order
    resultList = sortResults(resultList=resultList)
#     Convert the list of results back into a three-dimensional numpy array
    return np.stack(resultList,axis=2)

daskDelayedProcessingRaiseException(arr=arraySmall)

ValueError: Throw a ValueError to see how the libraries respond

## Conclusion
We see that `concurrent.futures` didn't raise an exception despite the `ValueError` in each iteration. Instead with both the threading and multiprocessing the function just returns `None`.

On the other hand `joblib` and `dask` did what we wanted and stopped execution of the parent function when the `ValueError` occurred. This example gives you a sense of the subtlties that emerge when you run parallel operations in production.